In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE = "https://www.federalreserve.gov"

def fetch_minutes_urls_historical_grouped(start_year=1993, end_year=2019):
    grouped_urls = {}

    for year in range(start_year, end_year + 1):
        url = f"{BASE}/monetarypolicy/fomchistorical{year}.htm"
        #print(f"📄 크롤링 중: {url}")

        res = requests.get(url)
        if res.status_code != 200:
            print(f"⚠️ {year} 페이지 접근 실패")
            continue

        soup = BeautifulSoup(res.text, "html.parser")

        year_urls = []

        # Minutes / Minutes of Actions 찾기
        for a in soup.find_all("a", href=True):
            href = a["href"].lower()

            if not href.endswith(".htm"):
                continue

            if ("minutes" in href) or ("actions" in href):
                full = urljoin(BASE, a["href"])
                year_urls.append(full)

        # 중복 제거 + 정렬
        grouped_urls[str(year)] = sorted(list(set(year_urls)))

    return grouped_urls


# ✅ 실행
grouped_minutes = fetch_minutes_urls_historical_grouped(1993, 2019)

# ✅ 연도별 개수 출력
for year, urls in grouped_minutes.items():
    print(year, ":", len(urls))

# ✅ 예시: 특정 연도 URL 보기
print("\n1993년 URL ㅡ>")
for u in grouped_minutes["1994"]:
    print(u)

In [ ]:
# ✅ 예시: 특정 연도 URL 보기
# 출력하고 싶은 연도 범위
start_year = 2011
end_year = 2019

for year in range(start_year, end_year + 1):
    year_str = str(year)
    if year_str in grouped_minutes:
        print(f"\n📌 {year}년 Minutes")
        for url in grouped_minutes[year_str]:
            print(url)
    else:
        print(f"\n⚠️ {year}년 데이터 없음")

In [ ]:
new_rows = []

for url in urls:
    print("\n처리 중:", url)

    # 날짜 추출
    date_match = re.search(r"(\d{8})", url)
    date = date_match.group(1) if date_match else "unknown"
    doc_id = f"FOMC_{date}"

    # ✅ 중복 방지
    if doc_id in existing_doc_ids:
        print(f"⛔ 이미 존재하는 문서 — 건너뜀: {doc_id}")
        continue

    raw_text, soup = fetch_minutes_text(url)

    title_tag = soup.find("h3")
    title = title_tag.get_text(strip=True) if title_tag else "FOMC Minutes"

    doc_type = "minutes"
    source = "Federal Reserve"
    language = "en"

    sentences = sent_tokenize(raw_text)

    doc_tokens = []
    doc_pos_tags = []
    doc_cleaned_sentences = []
    doc_lemmas = []

    for sent in sentences:
        tokens, pos_tags, cleaned, lemmas = preprocess_sentence(sent)
        doc_tokens.append(tokens)
        doc_pos_tags.append(pos_tags)
        doc_cleaned_sentences.append(cleaned)
        doc_lemmas.extend(lemmas)

    row = {
        "doc_id": doc_id,
        "doc_type": doc_type,
        "date": date,
        "title": title,
        "source": source,
        "language": language,
        # ✅ 줄바꿈 제거 → CSV 한 셀에 깔끔 저장
        "raw_text": raw_text.replace("\n", " ").replace("\r", " "),
        "sentences": json.dumps(sentences, ensure_ascii=False),
        "tokens": json.dumps(doc_tokens, ensure_ascii=False),
        "pos_tags": json.dumps(doc_pos_tags, ensure_ascii=False),
        "cleaned_sentences": json.dumps(doc_cleaned_sentences, ensure_ascii=False),
        "lemmas": json.dumps(doc_lemmas, ensure_ascii=False)
    }

    new_rows.append(row)